In [18]:
import pandas as pd
import logging 
import gc

In [3]:
# Resource
# https://www.kaggle.com/garethjns/microsoft-lightgbm-with-parameter-tuning-0-823
PATH = '/Users/pliu/Downloads/credit/'

In [4]:
logging.warning('loading data')
sub = pd.read_csv(PATH+'sample_submission.csv')
sub.head()

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5


In [5]:
# Feature Engineering

# bureau balance data
buro_bal = pd.read_csv(PATH+'bureau_balance.csv')
buro_bal.shape

(27299925, 3)

In [6]:
# get dummy - STATUS
buro_bal = pd.concat([buro_bal, pd.get_dummies(buro_bal.STATUS, prefix='buro_bal_status')],
                    axis=1).drop('STATUS', axis=1)
buro_bal.shape

(27299925, 10)

In [7]:
buro_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,buro_bal_status_0,buro_bal_status_1,buro_bal_status_2,buro_bal_status_3,buro_bal_status_4,buro_bal_status_5,buro_bal_status_C,buro_bal_status_X
0,5715448,0,0,0,0,0,0,0,1,0
1,5715448,-1,0,0,0,0,0,0,1,0
2,5715448,-2,0,0,0,0,0,0,1,0
3,5715448,-3,0,0,0,0,0,0,1,0
4,5715448,-4,0,0,0,0,0,0,1,0


In [17]:
# group by SK_ID_BUREAU
# frequency of SK_ID_BUREAU
buro_bal['buro_count'] = buro_bal.groupby('SK_ID_BUREAU')['SK_ID_BUREAU'].transform('count')
buro_bal.tail()

,SK_ID_BUREAU,MONTHS_BALANCE,buro_bal_status_0,buro_bal_status_1,buro_bal_status_2,buro_bal_status_3,buro_bal_status_4,buro_bal_status_5,buro_bal_status_C,buro_bal_status_X,buro_count
27299920,5041336,-47,0,0,0,0,0,0,0,1,24
27299921,5041336,-48,0,0,0,0,0,0,0,1,24
27299922,5041336,-49,0,0,0,0,0,0,0,1,24
27299923,5041336,-50,0,0,0,0,0,0,0,1,24
27299924,5041336,-51,0,0,0,0,0,0,0,1,24


In [19]:
del buro_count
gc.collect()

0

In [21]:
buro_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,buro_bal_status_0,buro_bal_status_1,buro_bal_status_2,buro_bal_status_3,buro_bal_status_4,buro_bal_status_5,buro_bal_status_C,buro_bal_status_X,buro_count
0,5715448,0,0,0,0,0,0,0,1,0,27
1,5715448,-1,0,0,0,0,0,0,1,0,27
2,5715448,-2,0,0,0,0,0,0,1,0,27
3,5715448,-3,0,0,0,0,0,0,1,0,27
4,5715448,-4,0,0,0,0,0,0,1,0,27


In [23]:
avg_buro_ball = buro_bal.groupby('SK_ID_BUREAU').mean()
avg_buro_ball.head()

,MONTHS_BALANCE,buro_bal_status_0,buro_bal_status_1,buro_bal_status_2,buro_bal_status_3,buro_bal_status_4,buro_bal_status_5,buro_bal_status_C,buro_bal_status_X,buro_count
SK_ID_BUREAU,,,,,,,,,,
5001709,-48.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,97.0
5001710,-41.0,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,83.0
5001711,-1.5,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,4.0
5001712,-9.0,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,19.0
5001713,-10.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,22.0


In [24]:
avg_buro_ball.columns = ['avg_buro_' + f for f in avg_buro_ball.columns]
avg_buro_ball.head()

,avg_buro_MONTHS_BALANCE,avg_buro_buro_bal_status_0,avg_buro_buro_bal_status_1,avg_buro_buro_bal_status_2,avg_buro_buro_bal_status_3,avg_buro_buro_bal_status_4,avg_buro_buro_bal_status_5,avg_buro_buro_bal_status_C,avg_buro_buro_bal_status_X,avg_buro_buro_count
SK_ID_BUREAU,,,,,,,,,,
5001709,-48.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,97.0
5001710,-41.0,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,83.0
5001711,-1.5,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,4.0
5001712,-9.0,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,19.0
5001713,-10.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,22.0


In [25]:
del buro_bal
gc.collect()

0

In [26]:
# Bureau data
buro = pd.read_csv(PATH+'bureau.csv')
print(buro.shape)
buro.head()

(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [27]:
buro_credit_active_dum = pd.get_dummies(buro.CREDIT_ACTIVE, prefix='ca_') 
buro_credit_currancy_dum = pd.get_dummies(buro.CREDIT_CURRENCY, prefix='cu_')
buro_credit_type_dum = pd.get_dummies(buro.CREDIT_TYPE, prefix='ty_')

In [28]:
con = [buro, buro_credit_active_dum, buro_credit_currancy_dum, buro_credit_type_dum]
buro_full = pd.concat(con, axis=1)
buro_full.shape

(1716428, 40)

In [29]:
del buro_credit_active_dum, buro_credit_currancy_dum, buro_credit_type_dum
gc.collect()

21

In [32]:
avg_buro_ball.reset_index().head()

,SK_ID_BUREAU,avg_buro_MONTHS_BALANCE,avg_buro_buro_bal_status_0,avg_buro_buro_bal_status_1,avg_buro_buro_bal_status_2,avg_buro_buro_bal_status_3,avg_buro_buro_bal_status_4,avg_buro_buro_bal_status_5,avg_buro_buro_bal_status_C,avg_buro_buro_bal_status_X,avg_buro_buro_count
0,5001709,-48.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,97.0
1,5001710,-41.0,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,83.0
2,5001711,-1.5,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,4.0
3,5001712,-9.0,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,19.0
4,5001713,-10.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,22.0


In [34]:
buro_full.shape

(1716428, 40)

In [35]:
# merge
buro_full = buro_full.merge(right=avg_buro_ball, how='left', on='SK_ID_BUREAU', suffixes=('', '_bur_bal'))

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,avg_buro_MONTHS_BALANCE,avg_buro_buro_bal_status_0,avg_buro_buro_bal_status_1,avg_buro_buro_bal_status_2,avg_buro_buro_bal_status_3,avg_buro_buro_bal_status_4,avg_buro_buro_bal_status_5,avg_buro_buro_bal_status_C,avg_buro_buro_bal_status_X,avg_buro_buro_count
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.500,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
buro_full.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,avg_buro_MONTHS_BALANCE,avg_buro_buro_bal_status_0,avg_buro_buro_bal_status_1,avg_buro_buro_bal_status_2,avg_buro_buro_bal_status_3,avg_buro_buro_bal_status_4,avg_buro_buro_bal_status_5,avg_buro_buro_bal_status_C,avg_buro_buro_bal_status_X,avg_buro_buro_count
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# counting buro per SK_ID_CURR
# count SK_ID_BUREAU
nb_bureau_per_curr = buro_full[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
buro_full['SK_ID_BUREAU'] = buro_full['SK_ID_CURR'].map(nb_bureau_per_curr['SK_ID_BUREAU'])

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,avg_buro_MONTHS_BALANCE,avg_buro_buro_bal_status_0,avg_buro_buro_bal_status_1,avg_buro_buro_bal_status_2,avg_buro_buro_bal_status_3,avg_buro_buro_bal_status_4,avg_buro_buro_bal_status_5,avg_buro_buro_bal_status_C,avg_buro_buro_bal_status_X,avg_buro_buro_count
0,215354,11,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,11,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,11,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,11,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,11,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
buro_full.head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,avg_buro_MONTHS_BALANCE,avg_buro_buro_bal_status_0,avg_buro_buro_bal_status_1,avg_buro_buro_bal_status_2,avg_buro_buro_bal_status_3,avg_buro_buro_bal_status_4,avg_buro_buro_bal_status_5,avg_buro_buro_bal_status_C,avg_buro_buro_bal_status_X,avg_buro_buro_count
0,215354,11,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,11,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,11,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,11,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,11,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,215354,11,Active,currency 1,-273,0,27460.0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,215354,11,Active,currency 1,-43,0,79.0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,162297,6,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,162297,6,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,162297,6,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
avg_buro = buro_full.groupby('SK_ID_CURR').mean()
avg_buro.head()

,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,...,avg_buro_MONTHS_BALANCE,avg_buro_buro_bal_status_0,avg_buro_buro_bal_status_1,avg_buro_buro_bal_status_2,avg_buro_buro_bal_status_3,avg_buro_buro_bal_status_4,avg_buro_buro_bal_status_5,avg_buro_buro_bal_status_C,avg_buro_buro_bal_status_X,avg_buro_buro_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,7.0,-735.000000,0.0,82.428571,-825.500000,NaN,0.0,207623.571429,85240.928571,0.00000,...,-11.785714,0.336651,0.007519,0.0,0.0,0.0,0.0,0.441240,0.214590,24.571429
100002,8.0,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.0,108131.945625,49156.200000,7997.14125,...,-21.875000,0.406960,0.255682,0.0,0.0,0.0,0.0,0.175426,0.161932,13.750000
100003,4.0,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.0,254350.125000,0.000000,202500.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,2.0,-867.000000,0.0,-488.500000,-532.500000,0.000,0.0,94518.900000,0.000000,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,3.0,-190.666667,0.0,439.333333,-123.000000,0.000,0.0,219042.000000,189469.500000,0.00000,...,-3.000000,0.735043,0.000000,0.0,0.0,0.0,0.0,0.128205,0.136752,7.000000


In [46]:
del buro, buro_full
gc.collect()

34

In [ ]:
# read previous applications
pd.read_csv(PATH+'')